In [1]:
# retrieve the preprocessed data from previous notebook

%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r le

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

Using TensorFlow backend.


In [3]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [4]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
__________

In [5]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 6985 samples, validate on 1747 samples
Epoch 1/100
6985/6985 [==============================] - 4s 583us/step - loss: 11.1227 - acc: 0.2165 - val_loss: 6.8835 - val_acc: 0.3526

Epoch 00001: val_loss improved from inf to 6.88353, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 2/100
6985/6985 [==============================] - 2s 352us/step - loss: 4.8051 - acc: 0.3087 - val_loss: 1.8028 - val_acc: 0.4139

Epoch 00002: val_loss improved from 6.88353 to 1.80277, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 3/100
6985/6985 [==============================] - 2s 336us/step - loss: 1.9797 - acc: 0.3576 - val_loss: 1.6131 - val_acc: 0.5243

Epoch 00003: val_loss improved from 1.80277 to 1.61305, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 4/100
6985/6985 [==============================] - 2s 335us/step - loss: 1.6974 - acc: 0.4283 - val_loss: 1.3788 - val_acc: 0.5736

Epoch 00004: val_loss improved from 1.61305 to 1.37876, saving m

In [6]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9252684323550465
Testing Accuracy:  0.8763594734511787


In [7]:
import librosa 
import numpy as np 

def extract_feature(file_name):
   
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None

    return np.array([mfccsscaled])


In [8]:
def print_prediction(file_name):
    prediction_feature = extract_feature(file_name) 

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

In [9]:
# Class: Air Conditioner

filename = '../UrbanSound Dataset sample/audio/100852-0-0-0.wav' 
print_prediction(filename) 

The predicted class is: air_conditioner 

air_conditioner 		 :  0.99989426136016845703125000000000
car_horn 		 :  0.00000715439318810240365564823151
children_playing 		 :  0.00001791530303307808935642242432
dog_bark 		 :  0.00000025655413082859013229608536
drilling 		 :  0.00000283992426375334616750478745
engine_idling 		 :  0.00005887898078071884810924530029
gun_shot 		 :  0.00000001620782441591472888831049
jackhammer 		 :  0.00000035662964137372910045087337
siren 		 :  0.00000004348472515403045690618455
street_music 		 :  0.00001830780820455402135848999023


In [10]:
# Class: Drilling

filename = '../UrbanSound Dataset sample/audio/103199-4-0-0.wav'
print_prediction(filename) 

The predicted class is: drilling 

air_conditioner 		 :  0.00000000003295356001964400149973
car_horn 		 :  0.00000000308959258177310402970761
children_playing 		 :  0.00002208830665040295571088790894
dog_bark 		 :  0.00000067401481373963179066777229
drilling 		 :  0.99972504377365112304687500000000
engine_idling 		 :  0.00000000002312424904338250541969
gun_shot 		 :  0.00000014346949228638550266623497
jackhammer 		 :  0.00000000029780389265710027757450
siren 		 :  0.00000000156893398273183493074612
street_music 		 :  0.00025209947489202022552490234375


In [11]:
# Class: Street music 

filename = '../UrbanSound Dataset sample/audio/101848-9-0-0.wav'
print_prediction(filename) 

The predicted class is: street_music 

air_conditioner 		 :  0.09999072551727294921875000000000
car_horn 		 :  0.00305506144650280475616455078125
children_playing 		 :  0.09950152784585952758789062500000
dog_bark 		 :  0.02582867257297039031982421875000
drilling 		 :  0.00509325042366981506347656250000
engine_idling 		 :  0.00916280318051576614379882812500
gun_shot 		 :  0.00549275847151875495910644531250
jackhammer 		 :  0.03270008042454719543457031250000
siren 		 :  0.00361734302714467048645019531250
street_music 		 :  0.71555775403976440429687500000000


In [12]:
# Class: Car Horn 

filename = '../UrbanSound Dataset sample/audio/100648-1-0-0.wav'
print_prediction(filename) 

The predicted class is: car_horn 

air_conditioner 		 :  0.00188611494377255439758300781250
car_horn 		 :  0.68632853031158447265625000000000
children_playing 		 :  0.01224335655570030212402343750000
dog_bark 		 :  0.16461659967899322509765625000000
drilling 		 :  0.05645351111888885498046875000000
engine_idling 		 :  0.00212736334651708602905273437500
gun_shot 		 :  0.00211420282721519470214843750000
jackhammer 		 :  0.00372551172040402889251708984375
siren 		 :  0.00587591761723160743713378906250
street_music 		 :  0.06462877988815307617187500000000


In [13]:
filename = '../Evaluation audio/dog_bark_1.wav'
print_prediction(filename) 

The predicted class is: dog_bark 

air_conditioner 		 :  0.00038618501275777816772460937500
car_horn 		 :  0.00915508810430765151977539062500
children_playing 		 :  0.06478454917669296264648437500000
dog_bark 		 :  0.71007812023162841796875000000000
drilling 		 :  0.02283692173659801483154296875000
engine_idling 		 :  0.00240809586830437183380126953125
gun_shot 		 :  0.10433794558048248291015625000000
jackhammer 		 :  0.00001514166433480568230152130127
siren 		 :  0.01288078445941209793090820312500
street_music 		 :  0.07311715185642242431640625000000


In [14]:
filename = '../Evaluation audio/drilling_1.wav'

print_prediction(filename) 

The predicted class is: drilling 

air_conditioner 		 :  0.32110649347305297851562500000000
car_horn 		 :  0.00000022923920539597020251676440
children_playing 		 :  0.00001040843835653504356741905212
dog_bark 		 :  0.00000026054382828988309483975172
drilling 		 :  0.66649377346038818359375000000000
engine_idling 		 :  0.00000000133662025891823077472509
gun_shot 		 :  0.00000043437574959170888178050518
jackhammer 		 :  0.01238841470330953598022460937500
siren 		 :  0.00000000002891160748308418959596
street_music 		 :  0.00000000528942090127770825347397


In [15]:
filename = '../Evaluation audio/gun_shot_1.wav'

print_prediction(filename) 

# sample data weighted towards gun shot - peak in the dog barking sample is simmilar in shape to the gun shot sample

The predicted class is: dog_bark 

air_conditioner 		 :  0.02008811198174953460693359375000
car_horn 		 :  0.00047429648111574351787567138672
children_playing 		 :  0.00094942341092973947525024414062
dog_bark 		 :  0.53654015064239501953125000000000
drilling 		 :  0.00093174201902002096176147460938
engine_idling 		 :  0.03123776055872440338134765625000
gun_shot 		 :  0.00091215252177789807319641113281
jackhammer 		 :  0.00002015420614043250679969787598
siren 		 :  0.00055970775429159402847290039062
street_music 		 :  0.40828645229339599609375000000000


In [16]:
filename = '../Evaluation audio/siren_1.wav'

print_prediction(filename) 

The predicted class is: siren 

air_conditioner 		 :  0.00000732402349967742338776588440
car_horn 		 :  0.00057092373026534914970397949219
children_playing 		 :  0.00199068244546651840209960937500
dog_bark 		 :  0.02090488374233245849609375000000
drilling 		 :  0.00046552356798201799392700195312
engine_idling 		 :  0.14164580404758453369140625000000
gun_shot 		 :  0.00050196843221783638000488281250
jackhammer 		 :  0.00276053301058709621429443359375
siren 		 :  0.81527197360992431640625000000000
street_music 		 :  0.01588040776550769805908203125000
